# Flower Recognition 

### Importing necessary libraries

In [ ]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
 
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

#preprocess.
from keras.preprocessing.image import ImageDataGenerator

#dl libraraies
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical

# specifically for cnn
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
 
import tensorflow as tf
import random as rn

# specifically for manipulating zipped images and getting numpy arrays of pixel values of images.
import cv2                  
import numpy as np  
from tqdm import tqdm
import os                   
from random import shuffle  
from zipfile import ZipFile
from PIL import Image

In [ ]:
!wget http://upscfever.com/datasets/flowers-new.zip

In [ ]:
!unzip flowers-new.zip


### Making the functions to get the training and validation set from the Images

In [ ]:
X=[]
Z=[]
IMG_SIZE=150
FLOWER_DAISY_DIR='flowers/daisy'
FLOWER_SUNFLOWER_DIR='flowers/sunflower'
FLOWER_TULIP_DIR='flowers/tulip'
FLOWER_DANDI_DIR='flowers/dandelion'
FLOWER_ROSE_DIR='flowers/rose'

In [ ]:
flower_dirs = [FLOWER_DAISY_DIR, FLOWER_SUNFLOWER_DIR, FLOWER_TULIP_DIR, FLOWER_DANDI_DIR, FLOWER_ROSE_DIR]

In [ ]:
flowers = ['Daisy', 'Sunflower', 'Tulip', 'Dandelion', 'Rose']

In [ ]:
def generate_images(flower, DIR, num=10):
  for img in tqdm(os.listdir(DIR)):
    img = cv2.imread(DIR + '/' + img)
    img = img.reshape((1,) + img.shape)

    gen_datagen = ImageDataGenerator(
          rotation_range=90,  
          zoom_range = 0.3, 
          width_shift_range=0.2,
          height_shift_range=0.2,
          horizontal_flip=True,
          vertical_flip=True,
          brightness_range=[0.5,1.5],
          shear_range=0.2
    )

    for idx, image_save in enumerate((gen_datagen.flow(img, 
                                                      batch_size=1, 
                                                      save_to_dir = DIR + '/', save_prefix=flower,
                                                      save_format='jpg'))):
      if idx == 30:
        break

In [ ]:
for flower, flower_dir in zip(flowers, flower_dirs):
  generate_images(flower, flower_dir)

In [ ]:
len(os.listdir(FLOWER_DAISY_DIR))

In [ ]:
def make_train_data(flower_type,DIR):
    for img in os.listdir(DIR):
        
        path = os.path.join(DIR,img)
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        
        X.append(np.array(img))
        Z.append(str(flower_type))

In [ ]:
for flower, flower_dir in zip(flowers, flower_dirs):
  make_train_data(flower, flower_dir)

In [ ]:
for flower_dir in flower_dirs:
  print(str(len(os.listdir(flower_dir))) + flower_dir)

In [ ]:
fig, ax = plt.subplots(5,5)
fig.set_size_inches(9,9)

for i in range(5):
  for j in range(5):

    ax[i][j].set_xticks([])
    ax[i][j].set_yticks([])

    l = rn.randint(0,len(Z))
    ax[i][j].imshow(X[l])
    ax[i][j].title.set_text('Flower: '+ Z[l])
    
plt.tight_layout()


In [ ]:
len(X)

In [ ]:
len(Z)

### Label Encoding the Y array (i.e. Daisy->0, Rose->1 etc...) & then One Hot Encoding

In [ ]:
le=LabelEncoder()
Y=le.fit_transform(Z)
Y=to_categorical(Y,5)
X=np.array(X)
X=X/255

### Splitting into Training and Validation Sets

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.25,random_state=42)

### Setting the Random Seeds

In [ ]:
np.random.seed(42)
rn.seed(42)


### Building the ANN

In [ ]:
import tensorflow as tf 
from tensorflow import keras 
import matplotlib.pyplot as plt 


In [ ]:
model = keras.Sequential([keras.layers.Flatten(input_shape=(150,150,3)),
                          keras.layers.Dense(128,activation = tf.nn.relu, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
                          keras.layers.Dense(64,activation = tf.nn.relu,kernel_regularizer=tf.keras.regularizers.l2(0.001)),
                          keras.layers.Dense(32,activation = tf.nn.relu,kernel_regularizer=tf.keras.regularizers.l2(0.001)),
                          keras.layers.Dense(5)])

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

### Building ANN Model 

In [ ]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

rd = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=55, verbose=1, mode='auto')
filepath='regularized_weight_{epoch:02d}_{val_loss:.2f}.h5'
mc = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

In [ ]:
history = model.fit(x_train,y_train, batch_size=100, validation_split=0.2, epochs = 100, verbose = 2, callbacks=[rd, es, mc])

In [ ]:
pred = model.evaluate(x_test, y_test)

#Add softmax to get predictions

In [ ]:
final_model = tf.keras.Sequential([
                                   model,
                                   tf.keras.layers.Softmax()
])

In [ ]:
predictions = final_model.predict_classes(x_test)

### Saving Model

In [ ]:
# save the model to disk
final_model.save('final_model2.h5')

### Loading Model

In [ ]:
from tensorflow import keras
model = keras.models.load_model('/content/regularized_weight_40_1.39.h5')


In [ ]:
model.summary()

#Deployment

In [ ]:
!pip install flask gevent requests pillow

Creat a file ProcFile

In [ ]:
procfile = 'web: gunicorn app:app'
procfiles= open("/content/Procfile","w")
procfiles.write(procfile)
procfiles.close()

#Install flask and ngrok

In [ ]:
!pip install flask-ngrok
from flask_ngrok import run_with_ngrok
from flask import Flask

#Webpage

In [ ]:
a = '''
<!doctype html>
<html lang="en">
<head>
<title>My flower recognizer</title>
</head> 
  <body>
 
          <h3>Image Recognition Server</h3><br/>
       
      <form action="" method="post" enctype=multipart/form-data>
        <input type=file name="flowers"><br/>
        
        <p>daisy = 0</p><br/>
        <p>dandelion = 1</p><br/>
        <p>rose = 2</p><br/>
        <p>sunflower = 3</p><br/>
        <p>tulip = 4</p><br/>
        <p>Enter flower number in text box</p><br/>
        <input type=text name="flower_name">

        <input type=submit value=Upload>
    </form>
    {{label}}  
     
  </body>
  </html>
  '''

In [ ]:
!mkdir '/content/templates'
!mkdir '/content/uploads'
!mkdir '/content/uploads/daisy'
!mkdir '/content/uploads/dandelion'
!mkdir '/content/uploads/rose'
!mkdir '/content/uploads/sunflower'
!mkdir '/content/uploads/tulip'

Html_file = open("/content/templates/index.html", "w")
Html_file.write(a)
Html_file.close()

#Deploy model

In [ ]:
import os
from flask import Flask, render_template, request
from keras.preprocessing import image
from keras.preprocessing.image import load_img, img_to_array

app = Flask(__name__)
run_with_ngrok(app)
app.config['UPLOADS'] = 'uploads'


def load_mymodel():
    global mymodel
    mymodel = keras.models.load_model('/content/regularized_weight_40_1.39.h5')
    final_model = tf.keras.Sequential([
                                   mymodel,
                                   tf.keras.layers.Softmax()
    ])
    

@app.route('/')
def home():
    return render_template('index.html')

def predictions(file):
    img = image.load_img(file, target_size=(150,150), color_mode="rgb")
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    ans = final_model.predict_classes(img)
    return ans


@app.route('/', methods=['POST'])
def upload_files():
    file = request.files['flowers']
    flowerslabels = request.form['flower_name']
    print(type(flowerslabels))
    if int(flowerslabels) == 0:
      flowerslabels = 'daisy'
      filepath = os.path.join(app.config['UPLOADS'], flowerslabels, file.filename)
      file.save(filepath)
    elif int(flowerslabels) == 1:
      flowerslabels = 'dandelion'
      filepath = os.path.join(app.config['UPLOADS'], flowerslabels, file.filename)
      file.save(filepath)
    elif int(flowerslabels) == 2:
      flowerslabels = 'rose'
      filepath = os.path.join(app.config['UPLOADS'], flowerslabels, file.filename)
      file.save(filepath)
    elif int(flowerslabels) == 3:
      flowerslabels = 'sunflower'
      filepath = os.path.join(app.config['UPLOADS'], flowerslabels, file.filename)
      file.save(filepath)
    else:
      flowerslabels = 'tulip'
      filepath = os.path.join(app.config['UPLOADS'], flowerslabels, file.filename)
      file.save(filepath)



    output = predictions(filepath)

    if output.astype('int32') == 0:
      output = 'daisy'
    elif output.astype('int32') == 1:
      output = 'dandelion'
    elif output.astype('int32') == 2:
      output = 'rose'
    elif output.astype('int32') == 3:
      output = 'sunflower'
    else:
      output = 'tulip'

    return render_template('index.html', label=output)



if __name__=='__main__':
    load_mymodel()
    app.run()

#Data augmentation

In [ ]:
#code part 1
from bs4 import BeautifulSoup
import numpy as np
import requests
import cv2
import PIL.Image
import urllib

cats_page = requests.get("http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=n11712282")# synset
print(cats_page.content)
cats_soup = BeautifulSoup(cats_page.content, 'html.parser')#puts the content of the website into the soup variable, each url on a different line


dogs_page = requests.get("http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=n12024176")# synset
print(dogs_page.content)
dogs_soup = BeautifulSoup(dogs_page.content, 'html.parser')#puts the content of the website into the soup variable, each url on a different line

dogs_page1 = requests.get("http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=n11971783")# synset
print(dogs_page1.content)
dogs_soup1 = BeautifulSoup(dogs_page1.content, 'html.parser')#puts the content of the website into the soup variable, each url on a different line
dogs_page2 = requests.get("http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=n11980318")# synset
print(dogs_page2.content)
dogs_soup2 = BeautifulSoup(dogs_page2.content, 'html.parser')#puts the content of the website into the soup variable, each url on a different line



#code part 2
cats_str_soup=str(cats_soup)#convert soup to string so it can be split
type(cats_str_soup)
cats_split_urls=cats_str_soup.split('\r\n')#split so each url is a different possition on a list
print(len(cats_split_urls))#print the length of the list so you know how many urls you have

#code part 2.2
dogs_str_soup=str(dogs_soup)#convert soup to string so it can be split
type(dogs_str_soup)
dogs_split_urls=dogs_str_soup.split('\r\n')#split so each url is a different possition on a list
print(len(dogs_split_urls))

dogs_str_soup1=str(dogs_soup1)#convert soup to string so it can be split
type(dogs_str_soup1)
dogs_split_urls1=dogs_str_soup1.split('\r\n')#split so each url is a different possition on a list
print(len(dogs_split_urls1))

dogs_str_soup2=str(dogs_soup2)#convert soup to string so it can be split
type(dogs_str_soup2)
dogs_split_urls2=dogs_str_soup2.split('\r\n')#split so each url is a different possition on a list
print(len(dogs_split_urls2))


img_rows, img_cols = 150, 150 #number of rows and columns to convert the images to
input_shape = (img_rows, img_cols, 3)#format to store the images (rows, columns,channels) called channels last

def url_to_image(url):
	# download the image, convert it to a NumPy array, and then read
	# it into OpenCV format
	resp = urllib.request.urlopen(url)
	image = np.asarray(bytearray(resp.read()), dtype="uint8")
	image = cv2.imdecode(image, cv2.IMREAD_COLOR)
 
	# return the image
	return image

n_of_training_images=20#the number of training images to use
for progress in range(n_of_training_images):#store all the images on a directory
 
    if(progress%20==0):
        print(progress)
    if not cats_split_urls[progress] == None:
      try:
        I = url_to_image(cats_split_urls[progress])
        if (len(I.shape))==3: #check if the image has width, length and channels
          save_path = '/content/uploads/tulip/tulip_'+str(progress)+'.jpg'#create a name of each image
          cv2.imwrite(save_path,I)

      except:
        None

for progress in range(n_of_training_images):#store all the images on a directory
  
    if(progress%20==0):
        print(progress)
    if not dogs_split_urls[progress] == None:
      try:
        I = url_to_image(dogs_split_urls[progress])
        if (len(I.shape))==3: #check if the image has width, length and channels
          save_path = '/content/uploads/dandelion/daisy2_'+str(progress)+'.jpg'#create a name of each image
          cv2.imwrite(save_path,I)

      except:
        None


for progress in range(n_of_training_images):#store all the images on a directory
  
    if(progress%20==0):
        print(progress)
    if not dogs_split_urls1[progress] == None:
      try:
        I = url_to_image(dogs_split_urls1[progress])
        if (len(I.shape))==3: #check if the image has width, length and channels
          save_path = '/content/uploads/daisy/daisy3_'+str(progress)+'.jpg'#create a name of each image
          cv2.imwrite(save_path,I)

      except:
        None


for progress in range(n_of_training_images):#store all the images on a directory
  
    if(progress%20==0):
        print(progress)
    if not dogs_split_urls2[progress] == None:
      try:
        I = url_to_image(dogs_split_urls2[progress])
        if (len(I.shape))==3: #check if the image has width, length and channels
          save_path = '/content/uploads/daisy/daisy4_'+str(progress)+'.jpg'#create a name of each image
          cv2.imwrite(save_path,I)

      except:
        None

In [ ]:
Xdevt=[]
Zdevt=[]
IMG_SIZE=150
FLOWER_DAISY_DIR='uploads/daisy'
FLOWER_SUNFLOWER_DIR='uploads/sunflower'
FLOWER_TULIP_DIR='uploads/tulip'
FLOWER_DANDI_DIR='uploads/dandelion'
FLOWER_ROSE_DIR='uploads/rose'

In [ ]:
flower_dirs = [FLOWER_DAISY_DIR, FLOWER_SUNFLOWER_DIR, FLOWER_TULIP_DIR, FLOWER_DANDI_DIR, FLOWER_ROSE_DIR]

In [ ]:
flowers = ['Daisy', 'Sunflower', 'Tulip', 'Dandelion', 'Rose']

In [ ]:
def generate_images(flower, DIR, num=10):
  for img in tqdm(os.listdir(DIR)):
    img = cv2.imread(DIR + '/' + img)
    img = img.reshape((1,) + img.shape)

    gen_datagen = ImageDataGenerator(
          rotation_range=90,  
          zoom_range = 0.3, 
          width_shift_range=0.2,
          height_shift_range=0.2,
          horizontal_flip=True,
          vertical_flip=True,
          brightness_range=[0.5,1.5],
          shear_range=0.2
    )

    for idx, image_save in enumerate((gen_datagen.flow(img, 
                                                      batch_size=1, 
                                                      save_to_dir = DIR + '/', save_prefix=flower,
                                                      save_format='jpg'))):
      if idx == 30:
        break

In [ ]:
for flower, flower_dir in zip(flowers, flower_dirs):
  generate_images(flower, flower_dir)

In [ ]:
len(os.listdir(FLOWER_DAISY_DIR))

In [ ]:
def make_train_data(flower_type,DIR):
    for img in os.listdir(DIR):
        
        path = os.path.join(DIR,img)
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        
        X.append(np.array(img))
        Z.append(str(flower_type))

In [ ]:
for flower, flower_dir in zip(flowers, flower_dirs):
  make_train_data(flower, flower_dir)

In [ ]:
for flower_dir in flower_dirs:
  print(str(len(os.listdir(flower_dir))) + flower_dir)

In [ ]:
fig, ax = plt.subplots(5,5)
fig.set_size_inches(9,9)

for i in range(5):
  for j in range(5):

    ax[i][j].set_xticks([])
    ax[i][j].set_yticks([])

    l = rn.randint(0,len(Z))
    ax[i][j].imshow(X[l])
    ax[i][j].title.set_text('Flower: '+ Z[l])
    
plt.tight_layout()


In [ ]:
len(Xdevt)

In [ ]:
len(Zdevt)

### Label Encoding the Y array (i.e. Daisy->0, Rose->1 etc...) & then One Hot Encoding

In [ ]:
le=LabelEncoder()
Ydevt=le.fit_transform(Zdevt)
Ydevt=to_categorical(Ydevt,5)
Xdevt=np.array(Xdevt)
Xdevt=Xdevt/255

### Splitting into Training and Validation Sets

In [ ]:
xdevt_train,xdevt_test,ydevt_train,ydevt_test=train_test_split(Xdevt,Ydevt,test_size=0.25,random_state=42)